# Volatility

In [ ]:
# %pip install pandas scipy requests matplotlib seaborn

In [ ]:
import re
import time
from time import sleep

import numpy as np
import requests
import math
import pandas as pd
from scipy.stats import norm

In [ ]:
API_KEY = {'X-API-Key': '114514'}
BASE_URL = "http://localhost:9999/v1"

s = requests.Session()
s.headers.update(API_KEY)

pd.set_option('display.max_columns', None)
pd.set_option('display.width', None)
pd.set_option('display.max_colwidth', None)
pd.set_option('display.expand_frame_repr', False)

In [ ]:
def bs(S, K, T, r, sigma, option_type):
    d1 = calculate_d1(sigma, S, K, T)
    d2 = calculate_d2(d1, sigma, T)

    if option_type == 'call':
        return S * norm.cdf(d1) - K * math.exp(-r * T) * norm.cdf(d2)
    elif option_type == 'put':
        return K * math.exp(-r * T) * norm.cdf(-d2) - S * norm.cdf(-d1)


def calculate_d1(sigma, S, K, T):
    try:
        # return (1 / (sigma * math.sqrt(T))) * (math.log(S / K) + (sigma ** 2 / 2) * T)
        return (math.log(S / K) + ( (sigma ** 2) / 2) * T) / (sigma * math.sqrt(T))
    except ZeroDivisionError:
        print(sigma, " ", S, " ", K, " ", T)


def calculate_d2(d1, sigma, T):
    return d1 - sigma * math.sqrt(T)


def implied_vol(S, K, T, r, price, option_type):
    vol = 0.22  # start value for the volatility
    d1 = calculate_d1(vol, S, K, T)
    test_price = bs(S, K, T, r, vol, option_type)  # BS price using the testing vol
    vega = S * math.sqrt(T) * norm.cdf(d1)  # vega of the option
    i = 0

    while True:
        vol = vol + (price - test_price) / vega
        if vol < 0.00001:
            vol = 0.00001
        d1 = calculate_d1(vol, S, K, T)
        vega = S * math.sqrt(T) * norm.cdf(d1)
        test_price = bs(S, K, T, r, vol, option_type)  # BS price using the testing vol

        diff = abs(price - test_price)
        i = i + 1

        if diff < abs(0.000001) or i > 200 or vol == 0.00001:
            break

    return vol


def delta_from_option(d1, position, option_type):
    if option_type == 'call':
        return norm.cdf(d1) * position * 100
    elif option_type == 'put':
        return (norm.cdf(d1) - 1) * position * 100
    else:
        raise Exception('Wrong option type')


def calculate_all_data(df, sec, realized_vol, stock_last_price, option_time_to_maturity):
    df['d1'] = df.apply(
        lambda row: calculate_d1(realized_vol, stock_last_price, row['strike'], option_time_to_maturity), axis=1)
    df['d2'] = df.apply(lambda row: calculate_d2(row['d1'], realized_vol, option_time_to_maturity), axis=1)
    df['d1'] = df['d1'].apply(lambda x: round(x, 4))
    df['d2'] = df['d2'].apply(lambda x: round(x, 4))

    # theoretical price
    df['thp'] = df.apply(
        lambda row: bs(stock_last_price, row['strike'], option_time_to_maturity, 0, realized_vol, row['type']), axis=1)
    df['thp'] = df['thp'].apply(lambda x: round(x, 2))

    # extract securities info
    df['last'] = sec['last'].reset_index(drop=True)
    df['position'] = sec['position'].reset_index(drop=True)

    # implied vol
    df['iv'] = df.apply(
        lambda row: implied_vol(stock_last_price, row['strike'], option_time_to_maturity, 0, row['last'], row['type']),
        axis=1)
    df['iv'] = df['iv'].apply(lambda x: round(x, 4))

    # d1 calculated with implied vol
    df['d1_iv'] = df.apply(
        lambda row: calculate_d1(row['iv'], stock_last_price, row['strike'], option_time_to_maturity), axis=1)

    # delta
    df['delta'] = df.apply(lambda row: delta_from_option(row['d1_iv'], row['position'], row['type']), axis=1)

    # df['vega'] = df.apply(
    #     lambda row: row['strike'] * math.sqrt(option_time_to_maturity) * norm.pdf(row['d1']) * (-1 if row["type"] == "put" else 1), axis=1)
    

In [ ]:
def parse_realized_vol(session):
    news = session.get(BASE_URL + '/news').json()

    if len(news) == 0:
        raise Exception('No news')

    if len(news) < 4:
        if news[-1]["ticker"] != "Delta Limit":
            return float(re.findall(r'\d+', news[-1]['body'])[1]) / 100
        else:
            return float(re.findall(r'\d+', news[-2]['body'])[1]) / 100

    if len(news) % 2 == 0:
        rv = float(re.findall(r'\d+',news[0]['body'])[0]) / 100
    else:
        rv = float(re.findall(r'\d+',news[1]['body'])[0]) / 100
        
    return rv


def parse_delta_limit(session):
    news = session.get(BASE_URL + '/news').json()

    if len(news) == 0:
        raise Exception('No news')
    
    for n in news:
        if n["headline"] == "Delta Limit":
            return int(re.findall(r'\d+', n['body'])[0]) * 1000

    raise Exception('No delta limit news')


def delta_hedge_by_stock(session, df, underlying_position, risk_exposure):
    delta = df['delta'].sum() + underlying_position
    print('Delta: ', delta)
    
    upper = risk_exposure
    lower = -risk_exposure

    # while delta >= upper or delta <= lower:
    if delta >= upper:
        post_order(session, 'MARKET', 3000, 'SELL', 'RTM')
        delta -= 5000
    elif delta <= lower:
        post_order(session, 'MARKET', 3000, 'BUY', 'RTM')
        delta += 5000
        


def post_order(s, type, quantity, action, ticker):
    r = s.post("http://localhost:9999/v1/orders?type="
            + type + "&quantity=" + str(quantity) + "&action=" + action + "&ticker=" + ticker)

    if r.status_code == 429:
        wait = r.json()["wait"]
        sleep(wait)

        post_order(s, type, quantity, action, ticker)
    elif r.status_code != 200:
        print(r.json())


def straddle(s, price, action, period):
    for _ in range(5):
        post_order(s, "MARKET", 100, action, "RTM" + str(period) + 'C' + str(price))
        post_order(s, "MARKET", 100, action, "RTM" + str(period) + 'P' + str(price))
        

In [ ]:
def calculate_macd(s, df):
    ema12 = df['iv'].ewm(span=12).mean()
    ema26 = df['iv'].ewm(span=26).mean()
    macd = ema12 - ema26
    signal = macd.ewm(span=12).mean()

    if macd.iloc[-1] > signal.iloc[-1]:
        return "BUY"
    elif macd.iloc[-1] < signal.iloc[-1]:
        return "SELL"
    else:
        return "HOLD"


def calculate_rsi(s, df):
    delta = df['iv'].diff()
    gain = delta.where(delta > 0, 0)
    loss = -delta.where(delta < 0, 0)

    # Calculate average gain and loss 
    average_gain = gain.rolling(window=15).mean()
    average_loss = loss.rolling(window=15).mean()

    # Calculate RSI 
    rs = average_gain / average_loss.abs()
    rsi = 100 - (100 / (1 + rs))
    return rsi.iloc[-1]


def close_position(s, ticker):
    pos = s.get("http://localhost:9999/v1/securities",params={"ticker": ticker}).json()[0]["position"]
    quantity = 10000 if ticker == "RTM" else 100

    while pos > 0:
        response = s.post(
            "http://localhost:9999/v1/orders",
            params={
                "ticker": ticker,
                "type": "MARKET",
                "action": "SELL",
                "quantity": min(quantity, pos),
            },
        )
        print("Closing position for", ticker, ":", response.json())
        pos -= min(quantity, pos)
    while pos < 0:
        response = s.post(
            "http://localhost:9999/v1/orders",
            params={
                "ticker": ticker,
                "type": "MARKET",
                "action": "BUY",
                "quantity": min(quantity, -pos),
            },
        )
        print("Closing position for ", ticker, " :", response.json())
        pos += min(quantity, -pos)

In [ ]:

def init_p1_data():
    df_1 = pd.DataFrame(columns=['name', 'type', 'strike'])
    df_1['type'] = ['call', 'put'] * 10
    df_1['name'] = ['RTM1C45' , 'RTM1P45', 'RTM1C46', 'RTM1P46', 'RTM1C47', 'RTM1P47', 'RTM1C48', 'RTM1P48', 'RTM1C49', 'RTM1P49', 'RTM1C50', 'RTM1P50', 'RTM1C51', 'RTM1P51', 'RTM1C52', 'RTM1P52', 'RTM1C53', 'RTM1P53', 'RTM1C54', 'RTM1P54']
    df_1['strike'] = [45, 45, 46, 46, 47, 47, 48, 48, 49, 49, 50, 50, 51, 51, 52, 52, 53, 53, 54, 54]

    return df_1


def init_p2_data():
    df_2 = pd.DataFrame(columns=['name', 'type', 'strike'])
    df_2['type'] = ['call', 'put'] * 10
    df_2['name'] = ['RTM2C45' , 'RTM2P45', 'RTM2C46', 'RTM2P46', 'RTM2C47', 'RTM2P47', 'RTM2C48', 'RTM2P48', 'RTM2C49', 'RTM2P49', 'RTM2C50', 'RTM2P50', 'RTM2C51', 'RTM2P51', 'RTM2C52', 'RTM2P52', 'RTM2C53', 'RTM2P53', 'RTM2C54', 'RTM2P54']
    df_2['strike'] = [45, 45, 46, 46, 47, 47, 48, 48, 49, 49, 50, 50, 51, 51, 52, 52, 53, 53, 54, 54]

    return df_2


from collections import deque

# Period 1:

In [ ]:
print('Period 1')
tick = s.get(BASE_URL + '/case').json()['tick']
risk_exposure = parse_delta_limit(s)
iv_queue = deque(maxlen=10)

holding_option_price = 50
print('Risk exposure: ', risk_exposure)

while 1 < tick < 260:
    period = s.get(BASE_URL + '/case').json()['period']
    if period == 2:
        raise Exception('Period 2')
    
    one_month_time_remaining = 300 - tick
    one_month_days_remaining = math.ceil(one_month_time_remaining / 30)
    one_month_option_time_to_maturity = one_month_days_remaining / 240

    df_1 = init_p1_data()
    
    realized_vol = parse_realized_vol(s)
    sec = pd.DataFrame(s.get(BASE_URL + '/securities').json())
    # drop other underlying
    stock_last_price = sec.at[0, 'last']
    underlying_position = sec.at[0, 'position']
    
    # drop first underlying
    sec = sec[1:]
    holding_position = sec['position'].sum() != 0
    
    if holding_position:
        price = int(sec[sec["position"] != 0]["ticker"].values[0][5:7])
        print("Holding position: ", price)
    else:
        price = round(stock_last_price)
    
    sec = sec[sec['ticker'].str.contains(str(price)) & sec['ticker'].str.contains('RTM1')].reset_index()
    df_1 = df_1[df_1['name'].str.contains(str(price))].reset_index()
    
    option_position = sec['position'].sum()
    
    d1 = calculate_d1(realized_vol, stock_last_price, price, one_month_option_time_to_maturity)
    d2 = calculate_d2(d1, realized_vol, one_month_option_time_to_maturity)

    iv_c = implied_vol(stock_last_price, price, one_month_option_time_to_maturity, 0, sec.at[0, 'last'], 'call')
    iv_p = implied_vol(stock_last_price, price, one_month_option_time_to_maturity, 0, sec.at[1, 'last'], 'put')
    iv = round((iv_c + iv_p) / 2, 4)
    # append only 
    if not iv_queue or iv_queue[-1] != iv:
        iv_queue.append(iv)
    # print(iv_queue)
    print("ATM IV: ", iv)
    
    if len(iv_queue) < 10:
        continue
        
    print("Mean: ", np.mean(iv_queue))
    print("STD: ", np.std(iv_queue))
    
    # if iv > np.mean(iv_queue) + 0.03:
    #     straddle(s, price, 'BUY', 1)
    #     if not holding_position:
    #         holding_position = True
    #         holding_option_price = price
    #     else:
    #         holding_position = False
    # elif iv < np.mean(iv_queue) - 0.03:
    #     straddle(s, price, 'SELL', 1)
    #     if not holding_position:
    #         holding_position = True
    #         holding_option_price = price
    #     else:
    #         holding_position = False

    
    d1_iv_c = calculate_d1(iv_c, stock_last_price, price, one_month_option_time_to_maturity)
    d1_iv_p = calculate_d1(iv_p, stock_last_price, price, one_month_option_time_to_maturity)
    
    delta_c = delta_from_option(d1_iv_c, sec.at[0, 'position'], 'call')
    delta_v = delta_from_option(d1_iv_p, sec.at[1, 'position'], 'put')

    delta = delta_c + delta_v + underlying_position
    print('Delta: ', delta)

    # while delta >= upper or delta <= lower:
    if delta >= risk_exposure:
        post_order(s, 'MARKET', 3000, 'SELL', 'RTM')
        delta -= 3000
    elif delta <= -risk_exposure:
        post_order(s, 'MARKET', 3000, 'BUY', 'RTM')
        delta += 3000
        
    tick = s.get(BASE_URL + '/case').json()['tick']
    # sleep(1)


# switch to 2 month option
sec = s.get(BASE_URL + '/securities', params={"ticker": "RTM1C50"}).json()[0]
if sec['position'] > 0:
    straddle(s, 50, 'SELL', 1)
elif sec['position'] < 0:
    straddle(s, 50, 'BUY', 1)
close_position(s, "RTM")

## Period 2:

In [ ]:
print('Period 2')
tick = s.get(BASE_URL + '/case').json()['tick']
period = s.get(BASE_URL + '/case').json()['period']
risk_exposure = parse_delta_limit(s)
iv_queue = deque(maxlen=10)

holding_position = False
holding_option_price = 50
print('Risk exposure: ', risk_exposure)

while 1 < tick < 299:
    period = s.get(BASE_URL + '/case').json()['period']
    
    if period == 1:
        time_remaining = 600 - tick
    else:
        time_remaining = 300 - tick
        
    days_remaining = math.ceil(time_remaining / 30)
    option_time_to_maturity = days_remaining / 240

    df_2 = init_p2_data()

    realized_vol = parse_realized_vol(s)
    sec = pd.DataFrame(s.get(BASE_URL + '/securities').json())
    # drop other underlying
    stock_last_price = sec.at[0, 'last']
    underlying_position = sec.at[0, 'position']

    # drop first underlying
    sec = sec[1:]
    holding_position = sec['position'].sum() != 0

    if holding_position:
        price = int(sec[sec["position"] != 0]["ticker"].values[0][5:7])
        print("Holding position: ", price)
    else:
        price = round(stock_last_price)

    sec = sec[sec['ticker'].str.contains(str(price)) & sec['ticker'].str.contains('RTM2')].reset_index()
    df_2 = df_2[df_2['name'].str.contains(str(price))].reset_index()

    option_position = sec['position'].sum()

    d1 = calculate_d1(realized_vol, stock_last_price, price, option_time_to_maturity)
    d2 = calculate_d2(d1, realized_vol, option_time_to_maturity)

    iv_c = implied_vol(stock_last_price, price, option_time_to_maturity, 0, sec.at[0, 'last'], 'call')
    iv_p = implied_vol(stock_last_price, price, option_time_to_maturity, 0, sec.at[1, 'last'], 'put')
    iv = round((iv_c + iv_p) / 2, 4)
    # append only 
    if not iv_queue or iv_queue[-1] != iv:
        iv_queue.append(iv)
    # print(iv_queue)
    print("ATM IV: ", iv)

    if len(iv_queue) < 10:
        continue

    print("Mean: ", np.mean(iv_queue))
    print("STD: ", np.std(iv_queue))


    # if iv > np.mean(iv_queue) + 0.03:
    #     straddle(s, price, 'BUY', 2)
    #     if not holding_position:
    #         holding_position = True
    #         holding_option_price = price
    #     else:
    #         holding_position = False
    # elif iv < np.mean(iv_queue) - 0.03:
    #     straddle(s, price, 'SELL', 2)
    #     if not holding_position:
    #         holding_position = True
    #         holding_option_price = price
    #     else:
    #         holding_position = False


    d1_iv_c = calculate_d1(iv_c, stock_last_price, price, option_time_to_maturity)
    d1_iv_p = calculate_d1(iv_p, stock_last_price, price, option_time_to_maturity)

    delta_c = delta_from_option(d1_iv_c, sec.at[0, 'position'], 'call')
    delta_v = delta_from_option(d1_iv_p, sec.at[1, 'position'], 'put')

    delta = delta_c + delta_v + underlying_position
    print('Delta: ', delta)

    # while delta >= upper or delta <= lower:
    if delta >= risk_exposure:
        post_order(s, 'MARKET', 3000, 'SELL', 'RTM')
        delta -= 3000
    elif delta <= -risk_exposure:
        post_order(s, 'MARKET', 3000, 'BUY', 'RTM')
        delta += 3000

    tick = s.get(BASE_URL + '/case').json()['tick']
    # sleep(1)


In [ ]:
sec = s.get(BASE_URL + '/securities').json()
for security in sec:
    if security['position'] != 0:
        close_position(s, security['ticker'])


In [ ]:
past = s.get(BASE_URL + '/securities', params={"ticker": "RTM1C49"}).json()
t = time.time()
while True:
    sec = s.get(BASE_URL + '/securities', params={"ticker": "RTM1C49"}).json()
    
    if past != sec:
        print("time: ", time.time() - t)
        t = time.time()
        past = sec

In [ ]:
import matplotlib.pyplot as plt
total_iv[total_iv['name'] == 'RTM2P50']['iv'].plot()